In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [20]:
import torchvision
import torchvision.transforms as transforms
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [21]:
import numpy as np
import matplotlib.pyplot as plt

In [22]:
train_set=torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',train=True,download=True,transform=transforms.Compose([transforms.ToTensor()])
)

In [23]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [24]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        #convulution layer
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        #linear layer
        self.fc1=nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2=nn.Linear(in_features=120,out_features=60)
        self.out=nn.Linear(in_features=60,out_features=10)


    def forward(self,t):
        #input layer
        t=t
        #conv layer
        t=self.conv1(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        t=self.conv2(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        #4th hidden layer
        t=t.reshape(-1,12*4*4)
        t=self.fc1(t)
        t=F.relu(t)
        #5th hidden layer
        t=self.fc2(t)
        t=F.relu(t)
        #6th layer
        t=self.out(t)
        return t


In [25]:
sample=next(iter(train_set))
image,label=sample

In [26]:
torch.set_grad_enabled(True)

In [27]:
net=Network()

In [33]:
#training loop
net=Network()
train_loader=torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)
optimizer=optim.Adam(net.parameters(),lr=0.01)
images,labels=next(iter(train_loader))
grid=torchvision.utils.make_grid(images)
tb=SummaryWriter("runs/mnist")
tb.add_image('images',grid)
tb.add_graph(net,images)
for epoch in range(10):
    total_loss=0
    total_correct=0
    for batch in train_loader:
        images,labels=batch
        preds=net(images)
        loss=F.cross_entropy(preds,labels)#claculate loss
        optimizer.zero_grad() #removing previous grad
        loss.backward() #calculationg grads
        optimizer.step() #updationg weight
        total_loss=total_loss+loss.item()
        total_correct=total_correct+get_num_correct(preds,labels)
    tb.add_scalar('loss',total_loss,epoch)
    tb.add_scalar('number correct',total_correct,epoch)
    tb.add_scalar('Accuracy',total_correct/len(train_set),epoch)

    tb.add_histogram('conv1.bias',net.conv1.bias,epoch)
    tb.add_histogram('conv1.weight',net.conv1.weight,epoch)
    tb.add_histogram('conv1.weight.grad',net.conv1.weight.grad,epoch)
    print("epoch:",epoch,"total_correct:",total_correct,"loss:",total_loss)
tb.close()

epoch: 0 total_correct: 47284 loss: 338.91982889175415
epoch: 1 total_correct: 51578 loss: 226.6000388264656
epoch: 2 total_correct: 52247 loss: 210.41048617661
epoch: 3 total_correct: 52681 loss: 198.47092899680138
epoch: 4 total_correct: 52906 loss: 191.41466762125492
epoch: 5 total_correct: 53064 loss: 188.6455678716302
epoch: 6 total_correct: 53271 loss: 185.32426212728024
epoch: 7 total_correct: 53298 loss: 181.26458413898945
epoch: 8 total_correct: 53541 loss: 177.83653269708157
epoch: 9 total_correct: 53610 loss: 174.43900357186794


In [32]:
for name,weight in net.named_parameters():
    print(name,weight.shape)

conv1.weight torch.Size([6, 1, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([12, 6, 5, 5])
conv2.bias torch.Size([12])
fc1.weight torch.Size([120, 192])
fc1.bias torch.Size([120])
fc2.weight torch.Size([60, 120])
fc2.bias torch.Size([60])
out.weight torch.Size([10, 60])
out.bias torch.Size([10])


In [37]:
total_correct/len(train_set)

0.8802666666666666

In [80]:
len(train_set.targets)

60000

In [90]:
#confusion matrix creation
def get_all_preds(model,loader):
    all_preds=torch.tensor([]) #empty ternsor
    for batch in loader:
        images,labels=batch
        preds=model(images)
        all_preds=torch.cat((all_preds,preds),dim=0)
    return all_preds



In [91]:
with torch.no_grad():
    prediction_loader=torch.utils.data.DataLoader(train_set,batch_size=10000)
    train_preds=get_all_preds(net,prediction_loader)

In [92]:
train_preds.shape

torch.Size([60000, 10])

In [93]:
print(train_preds.requires_grad)

False


In [95]:
preds_correct=get_num_correct(train_preds,train_set.targets)
print("total correct:",preds_correct)
print("accuracy:",preds_correct/len(train_set))


total correct: 52687
accuracy: 0.8781166666666667


In [96]:
train_set.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

In [97]:
train_preds.argmax(dim=1)

tensor([9, 0, 0,  ..., 3, 0, 5])

In [98]:
#confusion matrix
stacked=torch.stack(
    (
        train_set.targets,train_preds.argmax(dim=1)
    ),dim=1
)

In [99]:
stacked.shape

torch.Size([60000, 2])

In [100]:
stacked

tensor([[9, 9],
        [0, 0],
        [0, 0],
        ...,
        [3, 3],
        [0, 0],
        [5, 5]])

In [101]:
stacked[0].tolist()

[9, 9]

In [102]:
confusion_matrix=torch.zeros(10,10,dtype=torch.int32)
confusion_matrix

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)

In [104]:
for p in stacked:
    tl,pl=p.tolist()
    confusion_matrix[tl,pl]=confusion_matrix[tl,pl]+1

In [105]:
confusion_matrix

tensor([[5696,    5,   72,   73,   14,    6,   74,    0,   56,    4],
        [  17, 5841,    3,  110,    9,    3,    8,    0,    7,    2],
        [ 120,    0, 4819,   48,  672,   13,  263,    0,   60,    5],
        [ 270,   32,   26, 5384,  192,    1,   64,    0,   26,    5],
        [  30,    4,  472,  198, 4928,    2,  295,    1,   68,    2],
        [   0,    0,    0,    1,    0, 5807,    0,   72,   23,   97],
        [1896,    8,  623,  100,  499,    4, 2762,    0,  102,    6],
        [   0,    0,    0,    0,    0,   43,    0, 5735,   38,  184],
        [  27,    1,   21,   11,   17,    2,   19,    1, 5888,   13],
        [   0,    0,    0,    0,    0,   11,    0,  160,    2, 5827]], dtype=torch.int32)

In [106]:
train_preds

tensor([[-19.1264, -28.6650, -18.5898,  ...,   6.1554, -19.0720,  14.8729],
        [ 16.2407, -13.4272,  -0.8307,  ..., -30.2021,  -3.9194, -31.1329],
        [  4.9333,  -1.9516,  -0.6480,  ..., -13.1412,  -2.6298, -11.5398],
        ...,
        [  0.7022,   0.6432,  -4.0898,  ..., -13.7995,  -1.7056,  -9.2757],
        [  6.7638,  -6.8365,   0.9664,  ..., -20.9311,  -3.7574, -19.4840],
        [ -9.5916, -32.0981, -15.5527,  ...,   2.6363,  -5.5440,  -1.6642]])

In [14]:
from torch.utils.tensorboard import SummaryWriter

In [18]:
#tensorboard --logdir logs use this in conda cmd to open tenserboard
tb=SummaryWriter()
images,labels=next(iter(train_loader))
grid=torchvision.utils.make_grid(images)
tb.add_image('images',grid)
tb.add_graph(net,images)
tb.close()